In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0.3, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobtrain.applytojob.com/apply/N258gfMBOO/Supportive-Service-Specialist?source=Our%20Career%20Page%20Widget")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.






Supportive Service Specialist - JobTrain - Career Page



































 



Skip To Job Description





View All Jobs







View Our Website





















Welcome to the JobTrain Career Page!

Workforce development has never been more crucial to the vitality of the Bay Area economy than it is right now.  As we seek to revive from the impacts of COVID, low- and middle-income workers -- and those with barriers to employment -- need immediate and impactful assistance to stave off the hazards of homelessness, poverty and desperation. Our economy needs skilled workers at all levels, and JobTrain is the anti-poverty workforce development organization behind this important work.
 
JobTrain is an organization on the move.  We are delivering best in class outcomes, increasing our reach by expanding regionally, and we have committed to an ambitious new expanded mission to help our community to advance beyond the initial job placement, providing career advancement 

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [5]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Supportive Service Specialist',
  'experience': ['Direct experience working with specialized populations experiencing barriers to employment (i.e.,: people experiencing homelessness, low-income/public assistance, justice engaged, long-term unemployment, mature workers, DV survivors, etc.)',
   'Experience providing counseling services, ideally to adults.'],
  'skills': ['Ability to manage a caseload by assessing and meeting client needs, staying organized in follow up, documentation, and reporting.',
   'Excellent customer service and client rapport-building skills by maintaining and modeling a professional demeanor while efficiently performing the required job duties.',
   'Communication skills, especially the ability to guide conversations and achieve employment goals effectively & efficiently.',
   'Organization skills and attention to detail.',
   'Critical thinking through a lens of empathy, compassion, equity, innovation & inclusion.',
   'Understanding of grant funded

In [6]:
type(json_res)

list

In [7]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [8]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [9]:
links = collection.query(query_texts=['specialized populations experiencing barriers to employment, people experiencing homelessness'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}]]

In [10]:
job = json_res
job = ['skills']

In [12]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Tariq, a Monitoring, Evaluation & Research professional. Taba is an AI & Social Development Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous Communities out of poverty through our  tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Enhancing Supportive Services with AtliQ's Expertise

Dear Hiring Manager,

I came across the Supportive Service Specialist role at JobTrain and was impressed by the organization's commitment to empowering communities. As a Monitoring, Evaluation & Research professional at AtliQ, I'd like to introduce our company's capabilities in supporting your mission.

AtliQ is a seasoned AI & Social Development Consulting company with a proven track record of facilitating seamless integration of business processes through automated tools. Our tailored solutions have empowered numerous communities out of poverty, fostering scalability, process optimization, cost reduction, and heightened overall efficiency.

Our expertise in data management, analytics, and reporting can significantly enhance your Supportive Services program. We can help streamline client data management, provide actionable insights, and ensure compliance with grant-funded programs and reporting parameters.

Some of our not